# Electron Photon tagging

This is a first test to implement a QAE. 
The task is to identify Electrons and Photons from em-calorimeter images.

### imports 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
import tensorflow as tf
import tensorflow_quantum as tfq
from hep_VQAE import data_preprocessing as dp
from hep_VQAE import CAE as cae
from tensorflow.keras import layers, losses
import h5py
from sklearn.model_selection import train_test_split
from matplotlib.colors import LogNorm

import warnings
warnings.filterwarnings('ignore')

2022-08-03 19:22:42.174773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 19:22:42.174791: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/tom/.conda/envs/tfq/lib/python3.9/site-packages/cirq/ops/gateset.py:376: UserWarning: v0.14.1 is the last release `cirq.GlobalPhaseGate` is included by default. If you were relying on this behavior, you can include a `cirq.GlobalPhaseGate` in your `*gates`. If not, then you can ignore this warning. It will be removed in v0.16
  warnings.warn(
2022-08-03 19:22:43.994621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-03 19:22:43.994646: W tensorflow/

## data

In [3]:
f = h5py.File("../../data/electron.hdf5","r")
f2 = h5py.File("../../data/photon.hdf5","r")

In [4]:
electrons = f.get('X')[:,:,:,0]
photons = f2.get('X')[:,:,:,0]
electrons_y = f.get('y')[:]
photons_y = f2.get('y')[:]

In [5]:
x_train, x_val, y_train, y_val = train_test_split(np.vstack((electrons,photons)),
                                                    np.transpose(np.hstack((electrons_y, photons_y))),
                                                    test_size=0.2, shuffle=True)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                                    test_size=0.4, shuffle=True)

In [6]:
x_train_electrons = x_train[y_train==1]
x_train = x_train[y_train==0]

x_val_electrons = x_val[y_val==1]
x_val = x_val[y_val==0]

x_test_electrons = x_test[y_test==1]
x_test = x_test[y_test==0]

In [7]:
x_train = x_train.reshape(x_train.shape + (1,))
x_train_electrons = x_train_electrons.reshape(x_train_electrons.shape + (1,))

x_val = x_val.reshape(x_val.shape + (1,))
x_val_electrons = x_val_electrons.reshape(x_val_electrons.shape + (1,))

x_test = x_test.reshape(x_test.shape + (1,))
x_test_electrons = x_test_electrons.reshape(x_test_electrons.shape + (1,))

In [8]:
max_norm = np.max(x_train)
x_train = x_train / max_norm
x_train_electrons = x_train_electrons / max_norm

x_val = x_val / max_norm
x_val_electrons = x_val_electrons / max_norm

x_test = x_test / max_norm
x_test_electrons = x_test_electrons / max_norm

## keras tuner

In [31]:
tf.compat.v1.enable_eager_execution() 
from hep_VQAE import utils as ut
def emd(y_true, y_pred):
    avg_emd = ut.avg_emd(y_true.numpy(), y_pred.numpy())
    return avg_emd

In [32]:
def build_model(hp):
    latent_dim = hp.Int("latent_dim", min_value=40, max_value=200, step=20)
    fkc = hp.Choice('filters-kernels-choice',[0,1,2,3])
    if fkc == 0:
        filters, kernels = ([[8],[16],[32],[64]],[[],[],[],[]])
    elif fkc == 1:
        filters, kernels = ([[8,8],[16,16],[32],[64]],[[4],[4],[],[]])
    elif fkc == 2:
        filters, kernels = ([[16],[32],[64],[128]],[[],[],[],[]])
    elif fkc == 3:
        filters, kernels = ([[10,10,10],[10,10,10],[10,10,10],[10,10,10]],[[4,4],[4,4],[4,4,4],[2,2]])
    ae = cae.Convolutional_Autoencoder_hp_model(latent_dim,filters,kernels)
    lr = hp.Choice('lr', [0.01,0.001,0.0005,0.0001,0.00005])
    #ae.add_metric(emd, name='emd', aggregation='mean')
    ae.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr), run_eagerly=True, metrics=[emd])
    return ae

In [33]:
import keras_tuner
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective=keras_tuner.Objective("val_emd", direction="min"),
    max_trials=5,
    executions_per_trial=1,
    overwrite=True,
    directory="gammaetune",
    project_name="first_tune",
)

In [34]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
latent_dim (Int)
{'default': None, 'conditions': [], 'min_value': 40, 'max_value': 200, 'step': 20, 'sampling': None}
filters-kernels-choice (Choice)
{'default': 0, 'conditions': [], 'values': [0, 1, 2, 3], 'ordered': True}
lr (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0005, 0.0001, 5e-05], 'ordered': True}


In [40]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, x_train, epochs=5, steps_per_epoch=3, shuffle=True, batch_size=256, validation_data=(x_val[:5000], x_val[:5000]), callbacks=[es])


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
120               |120               |latent_dim
2                 |0                 |filters-kernels-choice
0.0001            |0.01              |lr

Epoch 1/5
3/3 [==============================] - 33s 15s/step - loss: 0.6907 - emd: 510.0371 - val_loss: 0.6887 - val_emd: 508.9910
Epoch 2/5
3/3 [==============================] - ETA: 0s - loss: 0.6875 - emd: 508.3708


KeyboardInterrupt



In [ ]:
tuner.results_summary()

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

# old emd metric stuff